# A6 – Motherhood Decline Analysis  
**Course**: Advanced Analytics & Dashboard Design  
**Project**: Achievement 6 – Independent Open Data Project  
**Author**: Yarisel Velacanto  
**Date**: March 2025

---

## Project Overview

This project explores the link between declining birth rates and the level of support (or lack of it) that governments provide for women, particularly around reproductive rights and family policies. The goal is to analyze trends in fertility, gender inequality, abortion rights, and child-rearing support to ask a critical question:

**Are women actually free to choose motherhood—or are they being pushed into it without the support they need?**

By combining datasets from global sources (Our World in Data, OECD, UNDP, WHO, World Bank, and GIWPS), this project builds a multi-layered view of how reproductive freedom, economic policy, and legal structures shape fertility outcomes worldwide.

#### Table of Contents

1. [Import Libraries & Set Display Options](#imports)
2. [Load and Preview Datasets](#load-preview)
3. [Data Cleaning & Preparation](#cleaning)
4. [Data Profile & Descriptive Statistics](#profile)
5. [Limitations & Ethical Considerations](#ethics)
6. [Defining Analytical Questions](#questions)

In [78]:
# Import Libraries & Set Display Options
# --------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Set display options for readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Optional: Styling for tables
from IPython.display import display, Markdown

---

## 2. Load and Preview Datasets <a id="load-preview"></a>

### Fertility Rate Dataset – Our World in Data

This dataset contains fertility rates by country over time, sourced from Our World in Data. It helps show where fertility is rising or falling and highlights broader trends in birth rates across regions and years.

---


In [55]:
# Load the Fertility Rate dataset
fertility_df = pd.read_csv('../02 Data/Original Data/children-born-per-woman/children-born-per-woman.csv')

# Preview the first few rows
fertility_df.head()

,Entity,Code,Year,"Fertility rate (period), historical"
0,Afghanistan,AFG,1950,7.25
1,Afghanistan,AFG,1951,7.26
2,Afghanistan,AFG,1952,7.26
3,Afghanistan,AFG,1953,7.27
4,Afghanistan,AFG,1954,7.25


### OECD Family Database – PF3.4: Childcare Support

This Excel file from the OECD Family Database includes three tabs: gross childcare fees, net cost for two-earner families, and net cost for single-parent families. It allows for comparison of the financial burden of childcare across family types and countries.

In [58]:
# Load the Gross Childcare Fees sheet
gross_fees_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Gross childcare fees'
)

# Preview the data
gross_fees_df.head()

,Country,Region,Childcare fee
0,Germany*,Berlin,1.00
1,Austria*,Vienna,3.00
2,Iceland*,Reykjavik,6.00
3,Latvia,NaN,7.00
4,Sweden,NaN,8.00


### Net Childcare Cost – Two-Earner Families

This sheet calculates the net cost of childcare as a percentage of average wages for two-earner households with children. It accounts for taxes, benefits, and direct costs, showing how affordable (or not) childcare is for working couples.

In [69]:
# Load the Net Cost for Two-Earner Families sheet
netcost_two_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Net cost two-earner familie'
)

# Preview the data
netcost_two_df.head()

,Country,Region,Childcare fee,Childcare benefit/rebates,Tax reduction,Changes in other benefits,Net cost (↗),"Net cost, % of family net income"
0,Malta,NaN,15.00,-15.00,0.00,0.00,0.00,0.00
1,Italy*,Rome,37.00,-37.00,0.00,0.00,0.00,0.00
2,Germany*,Berlin,1.00,0.00,0.00,0.00,1.00,1.00
3,Austria*,Vienna,3.00,0.00,0.00,0.00,3.00,2.00
4,Sweden,NaN,8.00,-3.00,0.00,0.00,5.00,4.00


### Net Childcare Cost – Single-Parent Families

This sheet shows the net cost of childcare for single-parent households, again as a percentage of average wages. It highlights how childcare affordability can differ drastically based on household structure, often placing a heavier burden on single parents.

In [72]:
# Load the Net Cost for Single-Parent Families sheet
netcost_single_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Net cost single-parent families'
)

# Preview the data
netcost_single_df.head()

,Country,Region,Childcare fee,Childcare benefit/rebates,Tax reduction,Changes in other benefits,Net cost (↗),"Net cost, % of family net income"
0,Canada*,Ontario,28.00,-22.00,-6.00,-1.00,-1.00,-1.00
1,Malta,NaN,15.00,-15.00,0.00,0.00,0.00,0.00
2,Portugal,NaN,21.00,-21.00,0.00,0.00,0.00,0.00
3,Greece,NaN,51.00,-51.00,0.00,0.00,0.00,0.00
4,Italy*,Rome,37.00,-37.00,0.00,0.00,0.00,0.00


### Global Abortion Policies Database – Abortion Policy Data

This dataset tracks whether countries have legal provisions for abortion across various legal frameworks (e.g., constitution, penal code, case law). Each legal source is coded as binary: 1 (legal provision exists), 0 (none). A composite flag is included to indicate whether abortion is recognized in **any** form. While this simplification doesn’t capture every nuance, it offers a useful signal of where abortion rights are legally acknowledged.

In [75]:
# Load Abortion Policy dataset
abortion_df = pd.read_excel(
    '../02 Data/Original Data/Abortion Policy Data-CLEAN.xlsx',
    sheet_name='Abortion_Law_Binary'
)

# Preview the data
abortion_df.head()

,Country,Reproductive Health Act,General Medical Health Act,Constitution,Criminal/ Penal Code,Civil Code,Ministerial Orders/ Decrees,Case Law,Health Regulation or Clinical Guideline,EML/ Registered List,Medical Ethics Code,Document relating to Funding,Abortion Specific Law,Law on Medical Practitioners,Law on Health Care Services,Other
0,Afghanistan,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1
1,Albania,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
2,Algeria,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,Andorra,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,Angola,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1


### UNDP Human Development Reports – Gender Inequality Index (GII)

The data highlights major gender gaps in education, labor participation, political representation, and maternal health. Key indicators related to men were kept to better emphasize the scale of inequality. Some cleaning was needed to remove footnotes and regional groupings before merging, but the final version offers a clear snapshot of how gender inequality differs across countries.

In [81]:
# Load Gender Inequality Index (GII) dataset
gii_df = pd.read_excel('../02 Data/Original Data/Gender Inequality Index (GII)-CLEAN.xlsx')

# Preview the data
gii_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,SDG3.1,SDG3.7,SDG5.5,SDG4.4,NaN,NaN,NaN
1,NaN,NaN,Gender Inequality Index,NaN,Maternal mortality ratio,Adolescent birth rate,Share of seats in parliament,Population with at least some secondary education,NaN,Labour force participation rate,NaN
2,NaN,NaN,Value,Rank,"(deaths per 100,000 live births)","(births per 1,000 women ages 15–19)",(% held by women),(% ages 25 and older),NaN,(% ages 15 and older),NaN
3,HDI rank,Country,NaN,NaN,NaN,NaN,NaN,Female,Male,Female,Male
4,NaN,NaN,2022,2022,2020,2022,2022,2022,2022,2022,2022


### Georgetown University – WPS Index

The WPS (Women, Peace, and Security) Index dives into how women experience safety, inclusion, and justice around the world. It includes variables on political voice, intimate partner violence, legal discrimination, and more. I removed regional category rows and standardized the country names to align with the rest of the datasets.

In [84]:
# Load WPS Index dataset
wps_df = pd.read_excel('../02 Data/Original Data/WPS Index-CLEAN.xlsx')

# Preview the cleaned data
wps_df.head()

,WPS Rank,Unnamed: 1,"Women, Peace and Security Index",Education,Employment,Financial Inclusion,Cell Phone Use,Parliamentary Representation,Absence of legal discrimination,Access to justice,Maternal mortality ratio,Son bias,Intimate partner violence,Community safety,Political violence targeting women,Proximity to conflict
0,2023,Country,2023,2016-2021,2018 - 2022,"2,021.00","2,022.00","2,023.00","2,023.00","2,022.00","2,020.00","2,022.00","2,018.00",2020-2022,"2,022.00",2021-2022
1,1,Denmark,0.93,13.16,76.96,100.00,100.00,43.58,100.00,3.96,4.66,105.70,3.00,78,0.00,0
2,2,Switzerland,0.93,13.48,78.44,99.02,91.00,39.34,88.12,3.89,7.38,105.10,2.00,85,0.00,0
3,3,Sweden,0.93,12.77,79.99,100.00,100.00,46.42,100.00,3.81,4.51,105.70,6.00,74,0.00,0
4,4,Finland,0.92,13.05,77.65,99.08,100.00,45.50,97.50,3.42,8.34,105.20,8.00,78,0.00,0


---

### Fertility Dataset – Initial Checks

We’ll start by reviewing the structure and contents of the fertility dataset. This includes checking data types, missing values, and value ranges to ensure the dataset is ready for analysis.

#### Basic Overview

In [88]:
# Check the structure
fertility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18958 entries, 0 to 18957
Data columns (total 4 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Entity                               18958 non-null  object 
 1   Code                                 17831 non-null  object 
 2   Year                                 18958 non-null  int64  
 3   Fertility rate (period), historical  18958 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 592.6+ KB


#### Summary Statistics

In [90]:
# View summary stats
fertility_df.describe()

,Year,"Fertility rate (period), historical"
count,"18,958.00","18,958.00"
mean,"1,985.85",3.95
std,22.06,2.00
min,"1,891.00",0.66
25%,"1,967.00",2.13
50%,"1,986.00",3.48
75%,"2,005.00",5.88
max,"2,023.00",8.86


#### Missing Values

In [95]:
# Check for missing values
fertility_df.isnull().sum()

Entity                                    0
Code                                   1127
Year                                      0
Fertility rate (period), historical       0
dtype: int64

---

### Fertility Dataset – Clean Column Names and Preview Data

Let’s rename the columns for clarity and consistency. We’ll also preview the data to ensure it looks correct.

In [99]:
# Rename columns for clarity
fertility_df.rename(columns={
    'Entity': 'Country',
    'Year': 'Year',
    'Total fertility rate (births per woman)': 'Fertility Rate'
}, inplace=True)

# Preview the cleaned dataframe
fertility_df.head()

,Country,Code,Year,"Fertility rate (period), historical"
0,Afghanistan,AFG,1950,7.25
1,Afghanistan,AFG,1951,7.26
2,Afghanistan,AFG,1952,7.26
3,Afghanistan,AFG,1953,7.27
4,Afghanistan,AFG,1954,7.25


### Fertility Dataset – Filter and Sort Data

We’ll keep only rows with non-null fertility rates and sort by year to prepare for time-series analysis.

In [106]:
fertility_df.rename(columns={'Fertility rate (period), historical': 'Fertility Rate'}, inplace=True)

In [108]:
# Drop rows with missing fertility rates
fertility_df = fertility_df.dropna(subset=['Fertility Rate'])

# Sort by country and year
fertility_df = fertility_df.sort_values(by=['Country', 'Year'])

# Reset index
fertility_df.reset_index(drop=True, inplace=True)

# Confirm changes
fertility_df.head()

,Country,Code,Year,Fertility Rate
0,Afghanistan,AFG,1950,7.25
1,Afghanistan,AFG,1951,7.26
2,Afghanistan,AFG,1952,7.26
3,Afghanistan,AFG,1953,7.27
4,Afghanistan,AFG,1954,7.25


---

## Step 3: Load and Clean OECD Childcare Support Data

This section focuses on preparing the OECD Family Database (PF3.4 – Childcare Support) for analysis.  
The goal is to reshape and clean the data from each of the three relevant sheets:

- Gross childcare fees  
- Net cost for two-earner families  
- Net cost for single-parent families  

We’ll begin by loading and transforming the **Gross Childcare Fees** sheet.

In [110]:
# Load the "Gross childcare fees" sheet
gross_fees_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Gross childcare fees'
)

# Preview the data
gross_fees_df.head()

,Country,Region,Childcare fee
0,Germany*,Berlin,1.00
1,Austria*,Vienna,3.00
2,Iceland*,Reykjavik,6.00
3,Latvia,NaN,7.00
4,Sweden,NaN,8.00


#### Clean and Reshape Gross Childcare Fees Data

In [113]:
# Drop any rows that are completely empty
gross_fees_df = gross_fees_df.dropna(how='all')

# Rename the first column to "Country"
gross_fees_df.rename(columns={gross_fees_df.columns[0]: "Country"}, inplace=True)

# Reshape from wide to long format
gross_fees_long = pd.melt(
    gross_fees_df,
    id_vars=["Country"],
    var_name="Year",
    value_name="Gross_Fee"
)

# Drop missing values
gross_fees_long = gross_fees_long.dropna()

# Preview the cleaned long-format data
gross_fees_long.head()

,Country,Year,Gross_Fee
0,Germany*,Region,Berlin
1,Austria*,Region,Vienna
2,Iceland*,Region,Reykjavik
8,Bulgaria*,Region,Sofia
14,Poland*,Region,Warsaw


---

## Step 3.3: Load and Clean Net Cost – Two-Earner Families

This sheet shows the **net childcare cost as a percentage of the average wage** for two-earner families.  
We'll load the data, rename the columns for consistency, and pivot the table to a long format so we can compare countries across years.

In [120]:
# Load the "Net cost two-earner families" sheet
net_cost_two_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Net cost two-earner familie'
)

# Preview the data
net_cost_two_df.head()

,Country,Region,Childcare fee,Childcare benefit/rebates,Tax reduction,Changes in other benefits,Net cost (↗),"Net cost, % of family net income"
0,Malta,NaN,15.00,-15.00,0.00,0.00,0.00,0.00
1,Italy*,Rome,37.00,-37.00,0.00,0.00,0.00,0.00
2,Germany*,Berlin,1.00,0.00,0.00,0.00,1.00,1.00
3,Austria*,Vienna,3.00,0.00,0.00,0.00,3.00,2.00
4,Sweden,NaN,8.00,-3.00,0.00,0.00,5.00,4.00


#### Reshape and Clean

In [123]:
# Rename the column for consistency
net_cost_two_df = net_cost_two_df.rename(columns={'Country': 'Country'})

# Reshape from wide to long format
net_cost_two_df = net_cost_two_df.melt(id_vars=['Country'], var_name='Year', value_name='Net Cost (Two-Earner)')

# Convert 'Year' to numeric
net_cost_two_df['Year'] = pd.to_numeric(net_cost_two_df['Year'], errors='coerce')

# Drop missing values
net_cost_two_df = net_cost_two_df.dropna(subset=['Net Cost (Two-Earner)'])

# Preview cleaned data
net_cost_two_df.head()

,Country,Year,Net Cost (Two-Earner)
1,Italy*,NaN,Rome
2,Germany*,NaN,Berlin
3,Austria*,NaN,Vienna
5,Iceland*,NaN,Reykjavik
11,Bulgaria*,NaN,Sofia


---

## Step 3.4: Load and Clean Net Cost – Single-Parent Families

This dataset captures the **net cost of childcare** for **single-parent households.** It helps us examine whether countries place disproportionate burdens on families with fewer resources. We'll load the sheet, reshape it for analysis, and clean up any formatting issues.

In [127]:
# Load the "Net cost single-parent families" sheet
net_cost_single_df = pd.read_excel(
    '../02 Data/Original Data/PF3-4-Childcare-support-CLEAN.xlsx',
    sheet_name='Net cost single-parent families'
)

# Preview the raw data
net_cost_single_df.head()

,Country,Region,Childcare fee,Childcare benefit/rebates,Tax reduction,Changes in other benefits,Net cost (↗),"Net cost, % of family net income"
0,Canada*,Ontario,28.00,-22.00,-6.00,-1.00,-1.00,-1.00
1,Malta,NaN,15.00,-15.00,0.00,0.00,0.00,0.00
2,Portugal,NaN,21.00,-21.00,0.00,0.00,0.00,0.00
3,Greece,NaN,51.00,-51.00,0.00,0.00,0.00,0.00
4,Italy*,Rome,37.00,-37.00,0.00,0.00,0.00,0.00


#### Reshape and Clean

In [130]:
# Rename the column for consistency
net_cost_single_df = net_cost_single_df.rename(columns={'Country': 'Country'})

# Reshape from wide to long format
net_cost_single_df = net_cost_single_df.melt(id_vars=['Country'], var_name='Year', value_name='Net Cost (Single-Parent)')

# Convert 'Year' to numeric
net_cost_single_df['Year'] = pd.to_numeric(net_cost_single_df['Year'], errors='coerce')

# Drop missing values
net_cost_single_df = net_cost_single_df.dropna(subset=['Net Cost (Single-Parent)'])

# Preview cleaned data
net_cost_single_df.head()

,Country,Year,Net Cost (Single-Parent)
0,Canada*,NaN,Ontario
4,Italy*,NaN,Rome
6,Germany*,NaN,Berlin
9,Japan*,NaN,Tokyo
12,Belgium*,NaN,Wallonie


---

## Step 4: Load and Clean Abortion Policy Dataset

Captures whether countries legally recognize abortion through various types of laws—such as criminal codes, health acts, or civil codes. The cleaned binary dataset will be loaded, and a simple composite flag created to show whether any form of abortion law exists in each country.

#### Code Block

In [135]:
# Load the abortion law binary dataset
abortion_df = pd.read_excel(
    '../02 Data/Original Data/Abortion Policy Data-CLEAN.xlsx',
    sheet_name='Abortion_Law_Binary'
)

# Preview the structure of the dataset
abortion_df.head()

,Country,Reproductive Health Act,General Medical Health Act,Constitution,Criminal/ Penal Code,Civil Code,Ministerial Orders/ Decrees,Case Law,Health Regulation or Clinical Guideline,EML/ Registered List,Medical Ethics Code,Document relating to Funding,Abortion Specific Law,Law on Medical Practitioners,Law on Health Care Services,Other
0,Afghanistan,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1
1,Albania,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
2,Algeria,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,Andorra,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,Angola,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1


#### Creating Composite Flag

In [138]:
# Create a composite flag if any law is present (i.e., sum of all columns > 0)
abortion_df['Abortion_Law_Flag'] = abortion_df.iloc[:, 1:].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# Preview the result
abortion_df[['Country', 'Abortion_Law_Flag']].head()

,Country,Abortion_Law_Flag
0,Afghanistan,1
1,Albania,1
2,Algeria,1
3,Andorra,1
4,Angola,1


---

## Step 5: Load and Clean Gender Inequality Index (GII)

The Gender Inequality Index (GII) dataset highlights disparities in education, labor force participation, and maternal health outcomes. We'll load the cleaned version of the dataset and keep key indicators—particularly those related to male labor and education—as a comparison baseline.

In [143]:
# Load Gender Inequality Index dataset
gii_df = pd.read_excel(
    '../02 Data/Original Data/Gender Inequality Index (GII)-CLEAN.xlsx'
)

# Preview the dataset
gii_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,SDG3.1,SDG3.7,SDG5.5,SDG4.4,NaN,NaN,NaN
1,NaN,NaN,Gender Inequality Index,NaN,Maternal mortality ratio,Adolescent birth rate,Share of seats in parliament,Population with at least some secondary education,NaN,Labour force participation rate,NaN
2,NaN,NaN,Value,Rank,"(deaths per 100,000 live births)","(births per 1,000 women ages 15–19)",(% held by women),(% ages 25 and older),NaN,(% ages 15 and older),NaN
3,HDI rank,Country,NaN,NaN,NaN,NaN,NaN,Female,Male,Female,Male
4,NaN,NaN,2022,2022,2020,2022,2022,2022,2022,2022,2022


---
## Step 6: Load and Clean WPS Index Dataset

Explores multiple dimensions of women’s safety, justice, and inclusion across countries. 
We’ll load the cleaned WPS Index data, rename the country column, and strip any whitespace 
so it's ready for merging with the other datasets.

In [155]:
# Load WPS Index data
wps_df = pd.read_excel(
    '../02 Data/Original Data/WPS Index-CLEAN.xlsx',
    sheet_name='WPS_Data'
)

# Rename column for countries
wps_df.rename(columns={'Unnamed: 1': 'Country'}, inplace=True)

# Optional: Clean column names and drop rows with no country name
wps_df.columns = wps_df.columns.str.strip()
wps_df = wps_df.dropna(subset=['Country'])
wps_df.reset_index(drop=True, inplace=True)

# Preview cleaned data
wps_df.head()

,WPS Rank,Country,"Women, Peace and Security Index",Education,Employment,Financial Inclusion,Cell Phone Use,Parliamentary Representation,Absence of legal discrimination,Access to justice,Maternal mortality ratio,Son bias,Intimate partner violence,Community safety,Political violence targeting women,Proximity to conflict
0,2023,Country,2023,2016-2021,2018 - 2022,"2,021.00","2,022.00","2,023.00","2,023.00","2,022.00","2,020.00","2,022.00","2,018.00",2020-2022,"2,022.00",2021-2022
1,1,Denmark,0.93,13.16,76.96,100.00,100.00,43.58,100.00,3.96,4.66,105.70,3.00,78,0.00,0
2,2,Switzerland,0.93,13.48,78.44,99.02,91.00,39.34,88.12,3.89,7.38,105.10,2.00,85,0.00,0
3,3,Sweden,0.93,12.77,79.99,100.00,100.00,46.42,100.00,3.81,4.51,105.70,6.00,74,0.00,0
4,4,Finland,0.92,13.05,77.65,99.08,100.00,45.50,97.50,3.42,8.34,105.20,8.00,78,0.00,0


---
## Step 7: Align Country Names Across All Datasets

Each dataset uses slightly different country names. 
To merge them successfully, we’ll create a master list of countries and standardize their names 
across all sources. This step ensures consistency for comparisons and analysis later on.

In [248]:
# Load the GII dataset with no skipped rows to inspect structure
gii_df_raw = pd.read_excel('../02 Data/Original Data/Gender Inequality Index (GII)-CLEAN.xlsx')
gii_df_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,SDG3.1,SDG3.7,SDG5.5,SDG4.4,NaN,NaN,NaN
1,NaN,NaN,Gender Inequality Index,NaN,Maternal mortality ratio,Adolescent birth rate,Share of seats in parliament,Population with at least some secondary education,NaN,Labour force participation rate,NaN
2,NaN,NaN,Value,Rank,"(deaths per 100,000 live births)","(births per 1,000 women ages 15–19)",(% held by women),(% ages 25 and older),NaN,(% ages 15 and older),NaN
3,HDI rank,Country,NaN,NaN,NaN,NaN,NaN,Female,Male,Female,Male
4,NaN,NaN,2022,2022,2020,2022,2022,2022,2022,2022,2022
5,1,Switzerland,0.02,3,7.38,2.20,39.02,96.94,97.52,61.49,71.94
6,2,Norway,0.01,2,1.66,2.19,44.97,99.09,99.27,62.53,69.59
7,3,Iceland,0.04,9,2.65,5.08,47.62,99.77,99.66,70.70,78.29
8,4,"Hong Kong, China (SAR)",-1,-1,-1,1.57,-1,77.87,84.07,52.91,64.71
9,5,Denmark,0.01,1,4.66,1.81,43.58,95.07,95.24,59.42,67.42


In [250]:
gii_df = pd.read_excel('../02 Data/Original Data/Gender Inequality Index (GII)-CLEAN.xlsx', header=4)

In [252]:
# Strip any leading/trailing whitespace
gii_df.columns = gii_df.columns.str.strip()

# Drop rows where 'Country' is missing
gii_df = gii_df[gii_df['Country'].notna()].reset_index(drop=True)

# Preview cleaned version
gii_df.head()


,HDI rank,Country,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Female,Male,Female.1,Male.1
0,1.00,Switzerland,0.02,3.00,7.38,2.20,39.02,96.94,97.52,61.49,71.94
1,2.00,Norway,0.01,2.00,1.66,2.19,44.97,99.09,99.27,62.53,69.59
2,3.00,Iceland,0.04,9.00,2.65,5.08,47.62,99.77,99.66,70.70,78.29
3,4.00,"Hong Kong, China (SAR)",-1.00,-1.00,-1.00,1.57,-1.00,77.87,84.07,52.91,64.71
4,5.00,Denmark,0.01,1.00,4.66,1.81,43.58,95.07,95.24,59.42,67.42


In [256]:
# Display current column names in wps_df
print(wps_df.columns)

# Rename 'Unnamed: 1' to 'Country' if that's where the names are
wps_df.rename(columns={'Unnamed: 1': 'Country'}, inplace=True)

print(wps_df[['Country']].head())

# Rebuild the country sets
wps_countries = set(wps_df['Country'].dropna().unique())

Index(['WPS Rank', 'Unnamed: 1', 'Women, Peace and Security Index',
       'Education', 'Employment', 'Financial Inclusion', 'Cell Phone Use',
       'Parliamentary Representation', 'Absence of legal discrimination',
       'Access to justice', 'Maternal mortality ratio', 'Son bias',
       'Intimate partner violence ', 'Community safety',
       'Political violence targeting women', 'Proximity to conflict'],
      dtype='object')
       Country
0      Country
1      Denmark
2  Switzerland
3       Sweden
4      Finland


In [258]:
# Drop the first row if it repeats column headers
wps_df = wps_df[wps_df['Country'] != 'Country'].reset_index(drop=True)

# Double-check it worked
print(wps_df[['Country']].head())

       Country
0      Denmark
1  Switzerland
2       Sweden
3      Finland
4      Iceland


In [260]:
# Build sets of country names from each dataset
fertility_countries = set(fertility_df['Country'].dropna().unique())
abortion_countries = set(abortion_df['Country'].dropna().unique())
gii_countries = set(gii_df['Country'].dropna().unique())
wps_countries = set(wps_df['Country'].dropna().unique())

# Compare missing countries
print("Missing from Abortion dataset:")
print(sorted(fertility_countries - abortion_countries))

print("\nMissing from GII dataset:")
print(sorted(fertility_countries - gii_countries))

print("\nMissing from WPS dataset:")
print(sorted(fertility_countries - wps_countries))

Missing from Abortion dataset:
['Africa (UN)', 'American Samoa', 'Anguilla', 'Aruba', 'Asia (UN)', 'Bermuda', 'Bonaire Sint Eustatius and Saba', 'British Virgin Islands', 'Brunei', 'Cape Verde', 'Cayman Islands', 'Congo', "Cote d'Ivoire", 'Curacao', 'Democratic Republic of Congo', 'East Timor', 'England & Wales', 'Eswatini', 'Ethiopia', 'Europe (UN)', 'Falkland Islands', 'Faroe Islands', 'French Guiana', 'French Polynesia', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 'High-income countries', 'Iran', 'Isle of Man', 'Israel', 'Jersey', 'Kosovo', 'Laos', 'Latin America and the Caribbean (UN)', 'Least developed countries', 'Less developed regions', 'Less developed regions, excluding China', 'Less developed regions, excluding least developed countries', 'Low-income countries', 'Lower-middle-income countries', 'Luxembourg', 'Macao', 'Martinique', 'Mayotte', 'Mexico', 'Micronesia (country)', 'Moldova', 'Montserrat', 'More developed regions', 'Myanmar', 'New Caledonia', 'North 

---
## Step 8: Merge All Datasets into a Single DataFrame

We'll begin by merging all the cleaned datasets using an outer join on the 'Country' column. This ensures we keep all countries, even if some have missing values in a specific dataset.

####  Create the Composite Abortion Law Flag

In [268]:
# Create a binary flag indicating if any abortion law exists
abortion_df['Abortion_Law_Flag'] = abortion_df.drop(columns=['Country']).notna().any(axis=1).astype(int)

# Preview the updated abortion_df
abortion_df[['Country', 'Abortion_Law_Flag']].head()

,Country,Abortion_Law_Flag
0,Afghanistan,1
1,Albania,1
2,Algeria,1
3,Andorra,1
4,Angola,1


#### Merge Net Cost

In [274]:
# Merge datasets one by one using outer join on 'Country'
merged_df = pd.merge(
    fertility_df[['Country', 'Year', 'Fertility Rate']],
    gross_fees_df[['Country', 'Childcare fee']],
    on='Country',
    how='outer'
)

# Merge Net Cost - Two-Earner
merged_df = pd.merge(
    merged_df,
    netcost_two_df[['Country', 'Net cost (↗)']],
    on='Country',
    how='left'
)
merged_df.rename(columns={'Net cost (↗)': 'Net Cost - Two-Earner'}, inplace=True)

# Merge Net Cost - Single-Parent
merged_df = pd.merge(
    merged_df,
    netcost_single_df[['Country', 'Net cost (↗)']],
    on='Country',
    how='left'
)
merged_df.rename(columns={'Net cost (↗)': 'Net Cost - Single-Parent'}, inplace=True)

#### Add Abortion Law Flag, GII, and WPS datasets to the merge

In [279]:
# Merge Abortion Policy Flag
merged_df = pd.merge(
    merged_df,
    abortion_df[['Country', 'Abortion_Law_Flag']],
    on='Country',
    how='left'
)

# Merge Gender Inequality Index (GII)
merged_df = pd.merge(
    merged_df,
    gii_df,
    on='Country',
    how='left'
)

# Merge Women, Peace & Security Index (WPS)
merged_df = pd.merge(
    merged_df,
    wps_df,
    on='Country',
    how='left'
)

# Preview final merged dataframe
merged_df.head()

,Country,Year,Fertility Rate,Childcare fee,Net Cost - Two-Earner,Net Cost - Single-Parent,Abortion_Law_Flag,HDI rank,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Female,Male,Female.1,Male.1,WPS Rank,"Women, Peace and Security Index",Education,Employment,Financial Inclusion,Cell Phone Use,Parliamentary Representation,Absence of legal discrimination,Access to justice,Maternal mortality ratio,Son bias,Intimate partner violence,Community safety,Political violence targeting women,Proximity to conflict
0,Afghanistan,"1,950.00",7.25,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
1,Afghanistan,"1,951.00",7.26,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
2,Afghanistan,"1,952.00",7.26,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
3,Afghanistan,"1,953.00",7.27,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
4,Afghanistan,"1,954.00",7.25,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12


## Step 9: Final Merged Dataset

All datasets have now been merged into one comprehensive DataFrame, capturing everything from birth rates and childcare costs to reproductive rights and gender inequality indicators. Below is a quick peek at the final structure to confirm that each column lined up correctly and that we have enough variables to analyze the relationships we care about.

By including countries that appear in any dataset (outer joins), we ensure our analysis can capture the widest range of conditions—though we’ll have to keep an eye on missing or patchy data along the way.

In [303]:
# Preview final merged dataframe
merged_df.head()

,Country,Year,Fertility Rate,Childcare fee,Net Cost - Two-Earner,Net Cost - Single-Parent,Abortion_Law_Flag,HDI rank,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Female,Male,Female.1,Male.1,WPS Rank,"Women, Peace and Security Index",Education,Employment,Financial Inclusion,Cell Phone Use,Parliamentary Representation,Absence of legal discrimination,Access to justice,Maternal mortality ratio,Son bias,Intimate partner violence,Community safety,Political violence targeting women,Proximity to conflict
0,Afghanistan,"1,950.00",7.25,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
1,Afghanistan,"1,951.00",7.26,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
2,Afghanistan,"1,952.00",7.26,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
3,Afghanistan,"1,953.00",7.27,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12
4,Afghanistan,"1,954.00",7.25,NaN,NaN,NaN,1.00,182.00,0.67,162.00,620.41,79.73,27.22,7.04,24.08,23.33,77.09,177,0.29,2.31,25.12,4.70,37.00,27.20,31.88,0.37,620.40,105.20,35.00,39,0.46,91.12


---
## Country Name Alignment Summary (What I Did)

In this analysis, we focused on aligning country names across multiple datasets related to various aspects of motherhood and gender equality. The datasets included fertility rates, childcare costs, net costs for both two-earner and single-parent families, abortion laws, and gender inequality indices. Below are the steps taken to ensure the country names were aligned across these sources:

1. **Standardization of Country Names:**
   - I ensured that country names were consistent across datasets by performing the necessary renaming and cleaning. This process involved standardizing any discrepancies, such as ensuring that countries with alternative spellings or formats matched across all data sources.

2. **Handling Missing Country Data:**
   - For any missing or mismatched country names, I reviewed each dataset, ensuring that valid country names were retained and any rows with missing country names were either cleaned or excluded from further analysis.

3. **Merge Datasets:**
   - I successfully merged all datasets using an outer join on the 'Country' column. This approach allowed for retaining all countries, even if some countries had missing data in a specific dataset.

4. **Review and Verification:**
   - A final check was performed to verify that all datasets now contained aligned country names, ensuring consistency across the entire merged dataset.

---

## Data Limitations (What's Still Missing)

While every effort was made to clean and merge the data, there are some limitations that should be noted:

1. **Missing or Incomplete Data:**
   - Some datasets had missing entries for certain countries, especially for variables like childcare fee, net cost, and certain political metrics. Despite this, an outer join was used to ensure no country was left out of the merged dataset, even if some data points were missing.

2. **Outdated Information:**
   - Certain datasets contained outdated information, particularly with respect to historical fertility rates and net costs. The data may not reflect the most recent policy changes or economic shifts in these countries.

3. **Inconsistent Data Sources:**
   - Some data sources used different formats or reported values in varying units. For example, the way net costs for two-earner families were calculated could differ from country to country, affecting direct comparability.

---

## Approach to Handle Missing or Mismatched Countries

1. **Data Cleaning:**
   - Any country name discrepancies were addressed by standardizing the names across datasets. For countries with missing or mismatched names, a manual review was conducted to match them to the correct country name.

2. **Imputation:**
   - Missing data for specific countries were handled by either excluding rows with significant gaps or using imputation techniques where feasible, although in this analysis, I focused on excluding rows with critical missing values rather than attempting to impute.

3. **Using Outer Joins:**
   - To ensure that all countries were included in the final dataset, outer joins were used during the merging process. This approach allowed for retaining all countries even if some had missing values in specific columns.

4. **Ongoing Verification:**
   - Regular checks were performed throughout the merging and cleaning process to identify any discrepancies or missing data that could affect the integrity of the analysis.

By following this process, I ensured that the final dataset was as complete and accurate as possible, with clear steps taken to handle any issues related to country names, missing data, or mismatched entries.

---

### Conclusion

The steps taken to align country names across all datasets were crucial in ensuring the quality and consistency of the final analysis. While some data limitations remain due to missing or inconsistent information, the overall approach allowed for a comprehensive exploration of the links between fertility rates, gender equality, and related factors such as childcare costs and political representation.

This analysis aims to provide insights into how societal systems shape women's choices, focusing on gaps in support and protections available to them. The next steps will involve deeper exploration of these patterns to inform recommendations for policy changes and areas of improvement.

